##### Copyright 2025 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Evaluating content safety with ShieldGemma 2 and Hugging Face Transformers

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/responsible/docs/safeguards/shieldgemma2_on_huggingface"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  </td>
    <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/responsible/docs/safeguards/shieldgemma2_on_huggingface.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/responsible/docs/safeguards/shieldgemma2_on_huggingface.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

The **ShieldGemma 2** model is trained to detect key harms detailed in the [model card](https://ai.google.dev/gemma/docs/shieldgemma/model_card_2). This guide demonstrates how to use Hugging Face Transformers to build robust data and models.

Note that `ShieldGemma 2` is trained to classify only one harm type at a time, so you will need to make a separate call to `ShieldGemma 2` for each harm type you want to check against. You may have additional that you can use model tuning techniques on `ShieldGemma 2`.

# Supported safety checks

**ShieldGemma2** is a model trained on Gemma 3's 4B IT checkpoint and is trained to detect and predict violations of key harm types listed below:

* **Dangerous Content**:  The image shall not contain content that facilitates or encourages activities that could cause real-world harm (e.g., building firearms and explosive devices, promotion of terrorism, instructions for suicide).

* **Sexually Explicit**: The image shall not contain content that depicts explicit or graphic sexual acts (e.g., pornography, erotic nudity, depictions of rape or sexual assault).

* **Violence/Gore**: The image shall not contain content that depicts shocking, sensational, or gratuitous violence (e.g., excessive blood and gore, gratuitous violence against animals, extreme injury or moment of death).

This serves as a foundation, but users can provide customized safety policies as input to the model, allowing for fine-grained control and specific use-case requirements.

# Supported Use Case

ShieldGemma 2 is should be used as an input filter to vision language models or as an output filter of image generation systems or both.**  ShieldGemma 2 offers the following key advantages:

* **Policy-Aware Classification**: ShieldGemma 2 accepts both a user-defined safety policy and an image as input, providing classifications for both real and generated images, tailored to the specific policy guidelines.
* **Probability-Based Output and Thresholding**: ShieldGemma 2 outputs a probability score for its predictions, allowing downstream users to flexibly tune the classification threshold based on their specific use cases and risk tolerance. This enables a more nuanced and adaptable approach to safety classification.

The input/output format are as follows:
* **Input**: Image + Prompt Instruction with policy definition
* **Output**: Probability of 'Yes'/'No' tokens, 'Yes' meaning that the image violated the specific policy. The higher the score for the 'Yes' token, the higher the model's confidence that the image violates the specified policy.

# Usage example

In [ ]:
# @title install Hugging Face Transformers v4.50+
! pip install -q 'transformers>=4.50.0'

In [ ]:
# @title Authenticate with Hugging Face Hub
# @markdown ShieldGemma is a gated model. To access the weights, you must accept
# @markdown the license on Hugging Face Hub under your account and then provide
# @markdown an [Access Token](https://huggingface.co/docs/hub/en/security-tokens)
# @markdown to authenticate with the Hugging Face Hub API. If using Colab, the
# @markdown easiest way to do this is by creating a read-only token specifically
# @markdown for Colab and setting this as the value of the `HF_TOKEN` secret;
# @markdown this token will then be reusable across all Colab notebooks. Other
# @markdown Python notebook platforms may provide a similar mechanism. For those
# @markdown that do not, un-comment the lines in this cell to install the
# @markdown Hugging Face Hub CLI and log in interactively.
# ! pip install -q 'huggingface_hub[cli]'
# ! huggingface-cli login

In [ ]:
from transformers import AutoProcessor, AutoModelForImageClassification
import torch

model_id = "google/shieldgemma-2-4b-it"

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForImageClassification.from_pretrained(model_id)
model.to(torch.device("cuda"))

In [ ]:
from PIL import Image
import requests

# The image included in this Colab is benign and will not violate any of
# ShieldGemma's built-in content policies. Change this URL or otherwise update
# this code to use an image that may be violative.
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
inputs = processor(images=[image], return_tensors="pt").to(torch.device("cuda"))

with torch.no_grad():
  scores = model(**inputs)

# `scores` is a `ShieldGemma2ImageClassifierOutputWithNoAttention` instance
# continaing the logits and probabilities associated with the model predicting
# the `Yes` or `No` tokens as the response to the prompt batch, captured in the
# following properties.
#
#   *   `logits` (`torch.Tensor` of shape `(batch_size, 2)`): The first position
#       along dim=1 is the logits for the `Yes` token and the second position
#       along dim=1 is the logits for the `No` token.
#   *   `probabilities` (`torch.Tensor` of shape `(batch_size, 2)`): The first
#       position along dim=1 is the probability of predicting the `Yes` token
#       and the second position along dim=1 is the probability of predicting the
#       `No` token.
#
# When used with the `ShieldGemma2Processor`, the `batch_size` will be equal to
# `len(images) * len(policies)`, and the order within the batch will be
# img1_policy1, ... img1_policyN, ... imgM_policyN.
print(scores.logits)
print(scores.probabilities)

# ShieldGemma prompts are constructed such that predicting the `Yes` token means
# the content violates the policy. If you are only interested in the violative
# condition, you can extract only that slice from the output tensors.
p_violated = scores.probabilities[:, 0]
print(p_violated)
